In [ ]:
!pip install torch transformers peft datasets
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 8.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [ ]:
# QLoRA 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load 4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=bnb_config,
    device_map="auto"
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Dataset
data = load_dataset("rajpurkar/squad_v2")

README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
import random

def format_with_citation(example):
    context = example["context"]
    question = example["question"]
    answer = example["answers"]["text"][0] if example["answers"]["text"] else "No answer"
    # Generating dummy citation
    chunk_id = random.randint(1, 100)
    page = random.randint(1, 50)
    timestamp = f"00:{random.randint(0,59):02}:{random.randint(0,59):02}"

    answer_with_citation = f"{answer} (chunk_{chunk_id}/page {page}/{timestamp})"

    input_text = f"""[INST] You are a helpful Q/A assistant.
Rules:
- Always answer in ≤3 sentences.
- Always include citation in this format: (chunk_id/page/timestamp).
- Never use "Answer:", "Short answer:", or bullet points.
- Output must be exactly one block of text: Answer + citation.

Here is the context:
```{context}```

Question: {question} [/INST]"""
    return {"input": input_text, "output": answer_with_citation}

subset = data["train"].select(range(5000))

# Split train/validation
split = subset.train_test_split(test_size=0.1, seed=42)

# Format rows
formatted_train = split["train"].map(format_with_citation)
formatted_val = split["test"].map(format_with_citation)



Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    # Tokenize input prompts
    inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",  # ensures tensors align
        max_length=128
    )

    # Tokenize outputs (answers)
    outputs = tokenizer(
        examples["output"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Labels = output_ids
    inputs["labels"] = outputs["input_ids"]
    return inputs

# Apply to dataset
tokenized_train = formatted_train.map(tokenize_function, batched=True, remove_columns=formatted_train.column_names)
tokenized_val = formatted_val.map(tokenize_function, batched=True, remove_columns=formatted_val.column_names)


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Add before trainer initialization
import gc
torch.cuda.empty_cache()
gc.collect()

103

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    num_train_epochs=1,
    warmup_ratio=0.03,
    fp16=True,
    logging_steps=10,
    eval_strategy="no",
    save_strategy="epoch",
    save_total_limit=1,
    dataloader_pin_memory=True,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,
)

model.config.use_cache = False
trainer.train()

# Save adapter
trainer.save_model("./mistral-7b-qlora-adapter-Q/A")
tokenizer.save_pretrained("./mistral-7b-qlora-adapter-Q/A")

# Save merged model
merged_model = model.merge_and_unload()
merged_model.save_pretrained("./mistral-7b-qlora-merged-Q/A")
tokenizer.save_pretrained("./mistral-7b-qlora-merged-Q/A")

print(" Training complete! Models saved:")


Step,Training Loss
10,3.029200
20,1.745800
30,0.945700
40,0.872700
50,0.846500
60,0.790200
70,0.779900
80,0.761500
90,0.718800
100,0.716000


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


 Training complete! Models saved:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A")


Mounted at /content/drive


('/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A/tokenizer_config.json',
 '/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A/special_tokens_map.json',
 '/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A/chat_template.jinja',
 '/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A/tokenizer.model',
 '/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A/added_tokens.json',
 '/content/drive/MyDrive/mistral-7b-qlora-merged-Q/A/tokenizer.json')

In [ ]:
test_dataset = load_dataset("rajpurkar/squad_v2", split="validation[:50]")

In [ ]:
formatted_test_dataset = test_dataset.map(format_with_citation)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.0
)


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
import re
import time

def has_citation(text):
    """Check if citation is present."""
    return bool(re.search(r"\(chunk.*?\)", text))

def count_sentences(text):
    """Count sentences (approx using . ? !)"""
    return len(re.findall(r"[.!?]", text))

def f1_score(prediction, ground_truth):
    """Compute F1 overlap between words."""
    pred_tokens = prediction.lower().split()
    gt_tokens = ground_truth.lower().split()
    common = set(pred_tokens) & set(gt_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    return 2 * precision * recall / (precision + recall)

def exact_match(prediction, ground_truth):
    return int(prediction.strip().lower() == ground_truth.strip().lower())

In [ ]:
import pandas as pd

results = []

for row in formatted_test_dataset:
    prompt = row["input"]
    gold_answer = row["output"]

    start = time.time()
    output = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]["generated_text"]
    latency = time.time() - start

    # Extract model's final answer
    answer = output.split("[/INST]")[-1].strip()

    # Checks
    citation_ok = has_citation(answer)
    em = exact_match(answer, gold_answer)
    f1 = f1_score(answer, gold_answer)
    length_ok = count_sentences(answer) <= 3

    results.append({
        "question": row["input"][:80] + "...",
        "gold": gold_answer,
        "pred": answer,
        "EM": em,
        "F1": round(f1, 3),
        "Citation": citation_ok,
        "Len<=3": length_ok,
        "Latency(s)": round(latency, 2),
    })

df = pd.DataFrame(results)

# Aggregate metrics
print("==== Aggregate Results ====")
print("Exact Match:", df["EM"].mean())
print("F1:", df["F1"].mean())
print("Citation compliance:", df["Citation"].mean())
print("Length compliance:", df["Len<=3"].mean())
print("Avg Latency:", df["Latency(s)"].mean())

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


==== Aggregate Results ====
Exact Match: 0.0
F1: 0.05724
Citation compliance: 0.94
Length compliance: 1.0
Avg Latency: 3.6096
